# Bulk Geocoding with Geopy and GeoPandas

## Overview

We will be working with an Excel sheet containing addresses of Hurricane Evacuation Centers in New York City. We will geocode these addresses and display the locations on a map.

Input Layers:

`Hurricane_Evacuation_Centers.xlsx`: A spreadsheet containing names and addresses of hurricane shelters inn NYC.

Output:

`hurricane_evacuation_centers.zip`: A zipped shapefile with geocoded locations of the hurricane evacuation centers.

Data Credit: NYC Open Data Portal. [source](https://data.cityofnewyork.us/Public-Safety/Hurricane-Evacuation-Centers-Map-/ayer-cga7)

> Note: This notebook has been updated to add a simpler (but less flexible) geocoding workflow using [`geopandas.tools.geocode`](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html) at the end.

---
A video walkthrough of the tutorial is available below

[![Watch Video Tutorial](https://img.youtube.com/vi/N6itC6hbOvo/0.jpg)](https://www.youtube.com/watch?v=N6itC6hbOvo)


## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [72]:
%%capture
if 'google.colab' in str(get_ipython()):
    !apt install libspatialindex-dev
    !pip install fiona shapely pyproj rtree mapclassify
    !pip install geopandas
    !pip install openpyxl
    !pip install leafmap

In [73]:
import os
import re
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from zipfile import ZipFile

In [74]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [75]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' +
         'Hurricane_Evacuation_Centers.xlsx')

## Procedure

Read the `Hurricane_Evacuation_Centers.xlsx` file using Pandas. This step requires the `openpyxl` package to be installed along with `pandas`. The dataset contains 64 addresses of hurricane evacuation centers in New York City.

In [96]:
excel_file = 'Hurricane_Evacuation_Centers.xlsx'
excel_file_path = os.path.join(data_folder, excel_file)
address_df = pd.read_excel(excel_file_path, skiprows=[0])
address_df

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE
0,Flushing,J.H.S. 185 - Queens,147-26 25 Drive,11354,4,NY,Y
1,Brooklyn,I.S. 258 - Brooklyn,141 Macon Street,11216,3,NY,Y
2,Brooklyn,I.S. 88 - Brooklyn,544 7 Avenue,11215,3,NY,Y
3,Bronx,Walton HS - X,2780 Reservoir Avenue,10468,2,NY,Y
4,Brooklyn,P.S./I.S.30 Mary White Ovington - Brooklyn,7002 4 Avenue,11209,3,NY,Y
...,...,...,...,...,...,...,...
59,Bronx,I.S. 201 - Bronx,730 Bryant Avenue,10474,2,NY,Y
60,Bronx,P.S. 46 - Bronx,2760 Briggs Avenue,10458,2,NY,Y
61,Long Island City,Aviation HS - Q,45-30 36 Street,11101,4,NY,Y
62,Bronx,P.S. 102 - Bronx,1827 Archer Street,10460,2,NY,Y


The Nominatim geocoder expects the addresses with certain format. We fix our source addresses by making the street numbers as ordinal numbers.

Examples

- `45-30 36 Street` → `45-30 36th Street`.
- `544 7 Avenue` → `544 7th Avenue`

In [97]:
def make_ordinal(match):
    n = int(match.group(1))
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix + match.group(2)

def update_address(row):
  old_address = row['ADDRESS']
  pattern = r'(\d+)(\s+(?:Street|Avenue|Blvd|Drive))'
  result = re.sub(pattern, make_ordinal, old_address)
  return result

address_df['ADDRESS_FIXED'] = address_df.apply(update_address, axis=1)
address_df

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE,ADDRESS_FIXED
0,Flushing,J.H.S. 185 - Queens,147-26 25 Drive,11354,4,NY,Y,147-26 25th Drive
1,Brooklyn,I.S. 258 - Brooklyn,141 Macon Street,11216,3,NY,Y,141 Macon Street
2,Brooklyn,I.S. 88 - Brooklyn,544 7 Avenue,11215,3,NY,Y,544 7th Avenue
3,Bronx,Walton HS - X,2780 Reservoir Avenue,10468,2,NY,Y,2780 Reservoir Avenue
4,Brooklyn,P.S./I.S.30 Mary White Ovington - Brooklyn,7002 4 Avenue,11209,3,NY,Y,7002 4th Avenue
...,...,...,...,...,...,...,...,...
59,Bronx,I.S. 201 - Bronx,730 Bryant Avenue,10474,2,NY,Y,730 Bryant Avenue
60,Bronx,P.S. 46 - Bronx,2760 Briggs Avenue,10458,2,NY,Y,2760 Briggs Avenue
61,Long Island City,Aviation HS - Q,45-30 36 Street,11101,4,NY,Y,45-30 36th Street
62,Bronx,P.S. 102 - Bronx,1827 Archer Street,10460,2,NY,Y,1827 Archer Street


We also create a new column containing the full address. The `CITY` column in the data are actually boroughs of NYC and adding it causing many geocoding requests to fail. Instead, we use the city name 'NYC' for all addresses.

In [98]:
address_df['Full_Address'] = (
    address_df['ADDRESS_FIXED'] + ',' +
    'NYC' + ',' +
    address_df['STATE']+ ',' +
    address_df['ZIP_CODE'].astype(str))
address_df

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE,ADDRESS_FIXED,Full_Address
0,Flushing,J.H.S. 185 - Queens,147-26 25 Drive,11354,4,NY,Y,147-26 25th Drive,"147-26 25th Drive,NYC,NY,11354"
1,Brooklyn,I.S. 258 - Brooklyn,141 Macon Street,11216,3,NY,Y,141 Macon Street,"141 Macon Street,NYC,NY,11216"
2,Brooklyn,I.S. 88 - Brooklyn,544 7 Avenue,11215,3,NY,Y,544 7th Avenue,"544 7th Avenue,NYC,NY,11215"
3,Bronx,Walton HS - X,2780 Reservoir Avenue,10468,2,NY,Y,2780 Reservoir Avenue,"2780 Reservoir Avenue,NYC,NY,10468"
4,Brooklyn,P.S./I.S.30 Mary White Ovington - Brooklyn,7002 4 Avenue,11209,3,NY,Y,7002 4th Avenue,"7002 4th Avenue,NYC,NY,11209"
...,...,...,...,...,...,...,...,...,...
59,Bronx,I.S. 201 - Bronx,730 Bryant Avenue,10474,2,NY,Y,730 Bryant Avenue,"730 Bryant Avenue,NYC,NY,10474"
60,Bronx,P.S. 46 - Bronx,2760 Briggs Avenue,10458,2,NY,Y,2760 Briggs Avenue,"2760 Briggs Avenue,NYC,NY,10458"
61,Long Island City,Aviation HS - Q,45-30 36 Street,11101,4,NY,Y,45-30 36th Street,"45-30 36th Street,NYC,NY,11101"
62,Bronx,P.S. 102 - Bronx,1827 Archer Street,10460,2,NY,Y,1827 Archer Street,"1827 Archer Street,NYC,NY,10460"


## Geocoding Method 1: Pandas

This is the preferred method that gives you more control over the geocoding process but involves a few more steps.

Now we use the [Nominatim Geocoder](https://geopy.readthedocs.io/en/stable/#nominatim) from GeoPy. Nominatim geocoder is based on OpenStreetMap data.

The geocoder is free to use but requires rate limiting our queries to 1 query per second. We use the `RateLimiter` provided by `geopy` package to set the delay. Since geocoding many address with the rate limit can take some time - we use a progress bar provided by the `tqdm` package to track the progress.

In [116]:
from tqdm.notebook import tqdm

tqdm.pandas()

locator = Nominatim(user_agent='spatialthoughts', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

address_df_pd = address_df.copy()
address_df_pd['location'] = address_df_pd['Full_Address'].progress_apply(geocode)
address_df_pd

  0%|          | 0/64 [00:00<?, ?it/s]

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE,ADDRESS_FIXED,Full_Address,location
0,Flushing,J.H.S. 185 - Queens,147-26 25 Drive,11354,4,NY,Y,147-26 25th Drive,"147-26 25th Drive,NYC,NY,11354","(Junior High School 185, 147-26, 25th Drive, L..."
1,Brooklyn,I.S. 258 - Brooklyn,141 Macon Street,11216,3,NY,Y,141 Macon Street,"141 Macon Street,NYC,NY,11216","(Junior High School 258, 141, Macon Street, Be..."
2,Brooklyn,I.S. 88 - Brooklyn,544 7 Avenue,11215,3,NY,Y,544 7th Avenue,"544 7th Avenue,NYC,NY,11215","(Middle School 88, 544, 7th Avenue, South Slop..."
3,Bronx,Walton HS - X,2780 Reservoir Avenue,10468,2,NY,Y,2780 Reservoir Avenue,"2780 Reservoir Avenue,NYC,NY,10468","(Celia Cruz High School of Music, 2780, Reserv..."
4,Brooklyn,P.S./I.S.30 Mary White Ovington - Brooklyn,7002 4 Avenue,11209,3,NY,Y,7002 4th Avenue,"7002 4th Avenue,NYC,NY,11209","(7002, 4th Avenue, Bay Ridge, Brooklyn, Kings ..."
...,...,...,...,...,...,...,...,...,...,...
59,Bronx,I.S. 201 - Bronx,730 Bryant Avenue,10474,2,NY,Y,730 Bryant Avenue,"730 Bryant Avenue,NYC,NY,10474","(730, Bryant Avenue, Hunts Point, Bronx County..."
60,Bronx,P.S. 46 - Bronx,2760 Briggs Avenue,10458,2,NY,Y,2760 Briggs Avenue,"2760 Briggs Avenue,NYC,NY,10458","(2760, Briggs Avenue, Bedford Park, Bronx Coun..."
61,Long Island City,Aviation HS - Q,45-30 36 Street,11101,4,NY,Y,45-30 36th Street,"45-30 36th Street,NYC,NY,11101","(45-30, 36th Street, Sunnyside, Queens, City o..."
62,Bronx,P.S. 102 - Bronx,1827 Archer Street,10460,2,NY,Y,1827 Archer Street,"1827 Archer Street,NYC,NY,10460","(Public School 102, 1827, Archer Street, Bronx..."


The response of the geocoding request is a [`geopy.location.Location`](https://geopy.readthedocs.io/en/stable/#geopy.location.Location) object containing the address along with geocoded coordinates as latitude and longitude. We extract them and add them to separate columns.

In [117]:
address_df_pd['latitude'] = address_df_pd['location'].apply(lambda loc: loc.latitude if loc else None)
address_df_pd['longitude'] = address_df_pd['location'].apply(lambda loc: loc.longitude if loc else None)
address_df_pd

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE,ADDRESS_FIXED,Full_Address,location,latitude,longitude
0,Flushing,J.H.S. 185 - Queens,147-26 25 Drive,11354,4,NY,Y,147-26 25th Drive,"147-26 25th Drive,NYC,NY,11354","(Junior High School 185, 147-26, 25th Drive, L...",40.774912,-73.818619
1,Brooklyn,I.S. 258 - Brooklyn,141 Macon Street,11216,3,NY,Y,141 Macon Street,"141 Macon Street,NYC,NY,11216","(Junior High School 258, 141, Macon Street, Be...",40.681918,-73.945597
2,Brooklyn,I.S. 88 - Brooklyn,544 7 Avenue,11215,3,NY,Y,544 7th Avenue,"544 7th Avenue,NYC,NY,11215","(Middle School 88, 544, 7th Avenue, South Slop...",40.660432,-73.987836
3,Bronx,Walton HS - X,2780 Reservoir Avenue,10468,2,NY,Y,2780 Reservoir Avenue,"2780 Reservoir Avenue,NYC,NY,10468","(Celia Cruz High School of Music, 2780, Reserv...",40.870595,-73.897498
4,Brooklyn,P.S./I.S.30 Mary White Ovington - Brooklyn,7002 4 Avenue,11209,3,NY,Y,7002 4th Avenue,"7002 4th Avenue,NYC,NY,11209","(7002, 4th Avenue, Bay Ridge, Brooklyn, Kings ...",40.633625,-74.024038
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Bronx,I.S. 201 - Bronx,730 Bryant Avenue,10474,2,NY,Y,730 Bryant Avenue,"730 Bryant Avenue,NYC,NY,10474","(730, Bryant Avenue, Hunts Point, Bronx County...",40.816040,-73.885264
60,Bronx,P.S. 46 - Bronx,2760 Briggs Avenue,10458,2,NY,Y,2760 Briggs Avenue,"2760 Briggs Avenue,NYC,NY,10458","(2760, Briggs Avenue, Bedford Park, Bronx Coun...",40.867779,-73.890245
61,Long Island City,Aviation HS - Q,45-30 36 Street,11101,4,NY,Y,45-30 36th Street,"45-30 36th Street,NYC,NY,11101","(45-30, 36th Street, Sunnyside, Queens, City o...",40.743475,-73.929464
62,Bronx,P.S. 102 - Bronx,1827 Archer Street,10460,2,NY,Y,1827 Archer Street,"1827 Archer Street,NYC,NY,10460","(Public School 102, 1827, Archer Street, Bronx...",40.838106,-73.865790


Some addresses might have failed to geocode. Let's review them.

In [118]:
failed = address_df_pd[address_df_pd['location'].isna()]
failed

,CITY,EC_Name,ADDRESS,ZIP_CODE,BOROCODE,STATE,ACCESSIBLE,ADDRESS_FIXED,Full_Address,location,latitude,longitude
54,New York,P.S. 48 - Manhattan,4360-78 Broadway,10033,1,NY,Y,4360-78 Broadway,"4360-78 Broadway,NYC,NY,10033",None,NaN,NaN


We may discard these rows. But sometimes it is useful to manually geocode failed queries. You may use another service manually or obtain the coordinates from another source. The below block assigns the coordinates 40.85339,-73.93350 to the row with index 54.

In [119]:
address_df_pd.loc[54, ['latitude', 'longitude']] = (40.85339,-73.93350)

Select a subset of columns to keep in the final result and rename them.

In [120]:
address_df_pd = address_df_pd[['EC_Name', 'Full_Address', 'latitude', 'longitude']]
address_df_pd.rename(columns = {'EC_Name': 'Name', 'Full_Address': 'Address'}, inplace=True)

We create a GeoDataFrame from the latitude and longitude columns.

In [121]:
geometry = gpd.points_from_xy(address_df_pd.longitude, address_df_pd.latitude)
address_gdf = gpd.GeoDataFrame(address_df_pd, crs='EPSG:4326', geometry=geometry)
address_gdf

,Name,Address,latitude,longitude,geometry
0,J.H.S. 185 - Queens,"147-26 25th Drive,NYC,NY,11354",40.774912,-73.818619,POINT (-73.81862 40.77491)
1,I.S. 258 - Brooklyn,"141 Macon Street,NYC,NY,11216",40.681918,-73.945597,POINT (-73.94560 40.68192)
2,I.S. 88 - Brooklyn,"544 7th Avenue,NYC,NY,11215",40.660432,-73.987836,POINT (-73.98784 40.66043)
3,Walton HS - X,"2780 Reservoir Avenue,NYC,NY,10468",40.870595,-73.897498,POINT (-73.89750 40.87059)
4,P.S./I.S.30 Mary White Ovington - Brooklyn,"7002 4th Avenue,NYC,NY,11209",40.633625,-74.024038,POINT (-74.02404 40.63362)
...,...,...,...,...,...
59,I.S. 201 - Bronx,"730 Bryant Avenue,NYC,NY,10474",40.816040,-73.885264,POINT (-73.88526 40.81604)
60,P.S. 46 - Bronx,"2760 Briggs Avenue,NYC,NY,10458",40.867779,-73.890245,POINT (-73.89025 40.86778)
61,Aviation HS - Q,"45-30 36th Street,NYC,NY,11101",40.743475,-73.929464,POINT (-73.92946 40.74347)
62,P.S. 102 - Bronx,"1827 Archer Street,NYC,NY,10460",40.838106,-73.865790,POINT (-73.86579 40.83811)


Visualize the geocoded addresses on a map.

In [122]:
import folium
m = leafmap.Map(width=800, height=500)
address_gdf.explore(
    m=m,
    marker_type='marker',
    marker_kwds={
        'icon': folium.Icon(color='#fdbb84', icon='hurricane', prefix='fa')
    }
)
m.zoom_to_gdf(address_gdf)
m

Save the results as a shapefile.

In [123]:
output_file = 'hurricane_evacuation_centers.shp'
output_path = os.path.join(output_folder, output_file)

address_gdf.to_file(filename=output_path)

We also zip the resulting shapefile along with all sidecar files so it can be downloaded easily.

In [124]:
output_zip_file = 'hurricane_evacuation_centers.zip'
output_zip_path = os.path.join(output_folder, output_zip_file)

sidecar_files = [
    os.path.join(output_folder, file)
    for file in os.listdir(output_folder)
    if file.endswith(('shp', 'shx', 'dbf', 'prj'))
]

with ZipFile(output_zip_path, 'w') as zip_object:
    for sidecar in sidecar_files:
      zip_object.write(sidecar, os.path.basename(sidecar))


## Geocoding Method 2: GeoPandas


GeoPandas also provides a helper tool [geopandas.tools.geocode](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html) for geocoding that allows using a GeoPy geocoder. This is a handy tool that gives you a GeoDataFrame but doesn't offer the flexibility of the previous method. We demonstrate the process here for the sake of completeness.

In [106]:
locator = Nominatim(user_agent='spatialthoughts', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

address_df_gpd = address_df.copy()

gdf = gpd.tools.geocode(address_df_gpd['ADDRESS_FIXED'])
gdf

,geometry,address
0,POINT (-73.81864 40.77489),"147-26, 25th Drive, 11354, 25th Drive, New Yor..."
1,POINT (-88.95624 39.84059),"Macon County Office Building, 141, South Main ..."
2,POINT (-122.19749 47.68078),"544, 7th Avenue, 98033, 7th Avenue, Kirkland, ..."
3,POINT (-73.89657 40.87026),"2780, Reservoir Avenue, 10468, Reservoir Avenu..."
4,POINT (-73.98978 40.67027),"4th Avenue, 10th Street, 11215, 10th Street, N..."
...,...,...
59,POINT (-73.88526 40.81604),"730, Bryant Avenue, 10474, Bryant Avenue, New ..."
60,POINT (-93.49384 34.93464),"Briggsville Avenue, 72828, Arkansas, United St..."
61,POINT (-73.92946 40.74347),"45-30, 36th Street, 11101, 36th Street, New Yo..."
62,POINT (-73.86579 40.83811),"Public School 102, 1827, Archer Street, 10460,..."


We merge the resulting GeoDataFrame with the original DataFrame to retain some columns from the original dataset.

In [ ]:
merged = gdf.join(address_df_gpd)
merged

We select the subset of columns and rename them.

In [109]:
merged = merged[['EC_Name', 'Full_Address', 'geometry']]
merged.rename(columns = {'EC_Name': 'Name', 'Full_Address': 'Address'}, inplace=True)
merged

,Name,Address,geometry
0,J.H.S. 185 - Queens,"147-26 25th Drive,NYC,NY,11354",POINT (-73.81864 40.77489)
1,I.S. 258 - Brooklyn,"141 Macon Street,NYC,NY,11216",POINT (-88.95624 39.84059)
2,I.S. 88 - Brooklyn,"544 7th Avenue,NYC,NY,11215",POINT (-122.19749 47.68078)
3,Walton HS - X,"2780 Reservoir Avenue,NYC,NY,10468",POINT (-73.89657 40.87026)
4,P.S./I.S.30 Mary White Ovington - Brooklyn,"7002 4th Avenue,NYC,NY,11209",POINT (-73.98978 40.67027)
...,...,...,...
59,I.S. 201 - Bronx,"730 Bryant Avenue,NYC,NY,10474",POINT (-73.88526 40.81604)
60,P.S. 46 - Bronx,"2760 Briggs Avenue,NYC,NY,10458",POINT (-93.49384 34.93464)
61,Aviation HS - Q,"45-30 36th Street,NYC,NY,11101",POINT (-73.92946 40.74347)
62,P.S. 102 - Bronx,"1827 Archer Street,NYC,NY,10460",POINT (-73.86579 40.83811)
